## 数据准备

首先，我们需要准备训练数据，我们有以下选择：

1. 使用sklearn中的手写数字（8*8）；
2. 使用 [tfds](https://github.com/tensorflow/datasets/tree/master) 中的手写数字(28*28) [mnist](https://www.tensorflow.org/datasets/catalog/mnist)。

sklean 中的数字的分辨率比较差，人工识别可能都有困难；我们使用 tfds 中的 mnist 数据集。

另外，tfds 使用手册可见 [TensorFlow Datasets](https://www.tensorflow.org/datasets/overview?hl=zh-cn)。

In [ ]:
from sklearn import datasets
from matplotlib import pyplot

# sklearn中的手写数据集
digits = datasets.load_digits()

pyplot.gray()
pyplot.matshow(digits.images[8])
# pyplot.matshow(digits.images[16])
pyplot.show()

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt

# tensorflow中的手写数据集
mnist, info = tfds.load("mnist", split="train", shuffle_files=True, with_info=True)

# 通过pyplot呈现
# plt.gray()
# for example in tfds.as_numpy(mnist.take(3)):
#     print(example.keys())
#     plt.matshow(example["image"])

# 通过pandas.DataFrame呈现
# tfds.as_dataframe(mnist.take(5), info)

# 调用tfds.show_examples呈现
tfds.show_examples(mnist.take(2), info)

## 通过深度学习实现手写数字识别

### 通过keras API实现关于手写数字识别的深度学习

主要步骤是：

1. 通过 `keras.Sequential` 和 `keras.layers.Dense` 创建模型。需要指定模型的层数、每层的节点数以及每层的激活函数。
2. 为模型指定损失函数和优化器。损失函数用于衡量模型预测值偏离实际值的程度，而优化器则用于实现训练过程。
3. 利用训练数据对模型进行训练。

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt

# 加载训练数据和测试数据
(orig_train_images, train_labels), (orig_test_images, test_labels) = keras.datasets.mnist.load_data()
xx = np.array([])
assert(orig_train_images.shape == (60000, 28, 28))
assert(train_labels.shape == (60000,))
assert(orig_test_images.shape == (10000, 28, 28))
assert(test_labels.shape == (10000,))
train_images = orig_train_images.reshape(60000, 28*28)
train_images = train_images.astype("float32") / 255
test_images = orig_test_images.reshape(10000, 28*28)
test_images = test_images.astype("float32") / 255

# 构建深度学习模型
model = keras.Sequential([
    keras.layers.Dense(512, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

# 编译，即：为模型指定损失函数和优化器
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

# 训练
model.fit(train_images[0:10000], train_labels[0:10000], epochs=5, batch_size=128)

# 预测
start_idx = np.random.randint(0, 10000-10)
predictions_as_prop = model.predict(test_images[start_idx:start_idx+10])
predictions = []
for a in predictions_as_prop:
    predictions.append(np.argmax(a))

plt.gray()
fig, axes = plt.subplots(2, 5)
fig.set_size_inches(5*2.5, 2*2.5)
for i in range(10):
    axes[i//5, i%5].matshow(orig_test_images[start_idx+i])
plt.show()

print("%+12s:"%"predictions", np.array(predictions))
print("%+12s:"%"real", test_labels[start_idx:start_idx+10])

### 基于TensorFlow从头开始实现深度学习



In [ ]:
from typing import Any
import numpy as np
import tensorflow as tf

# 简单的层对象的实现
class SimpleDense:
    def __init__(self, input_size, output_size, activation) -> None:
        self.activation = activation

        # 通过随机值来初始化权重(weight)
        initail_value = [[np.random.uniform(0., 1.) for j in range(output_size)] for i in range(input_size)]
        self.W = tf.Variable(initail_value)

        # 初始化偏置(bias)
        self.b = tf.Variable(np.zeros((output_size,)))

    def __call__(self, input: tf.Variable) -> tf.Variable:
        return self.activation(np.dot(self.W, input) + self.b)

d = SimpleDense(10, 4, tf.nn.softmax)
print(d)